In [5]:
import sys
import os
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob


import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import clear_output
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

UsageError: Line magic function `%` not found.


In [6]:
#@title Enter Twitter Credentials
TWITTER_KEY = 'HTi8RVQq3LczryTEyESQBdqNk' #@param {type:"string"}
TWITTER_SECRET_KEY = '5VYQAWZTLuBs2p9mbLmkoGvCg7iR0vnXvKMdEDyEGcSKS5jcQk' #@param {type:"string"}

In [7]:
# Authenticate
auth = tweepy.AppAuthHandler(TWITTER_KEY, TWITTER_SECRET_KEY)

api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [31]:
#@title Twitter Search API Inputs
#@markdown ### Enter Search Query:
searchQuery = 'Karachi' #@param {type:"string"}
#@markdown ### Enter Max Tweets To Scrape:
#@markdown #### The Twitter API Rate Limit (currently) is 45,000 tweets every 15 minutes.
maxTweets = 100 #@param {type:"slider", min:0, max:45000, step:100}
Filter_Retweets = True #@param {type:"boolean"}

tweetsPerQry = 100  # this is the max the API permits
tweet_lst = []

if Filter_Retweets:
  searchQuery = searchQuery + ' -filter:retweets'  # to exclude retweets

# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -10000000000

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
while tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, lang="en")
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", max_id=str(max_id - 1),
                                        since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
            break
        for tweet in new_tweets:
          if hasattr(tweet, 'reply_count'):
            reply_count = tweet.reply_count
          else:
            reply_count = 0
          if hasattr(tweet, 'retweeted'):
            retweeted = tweet.retweeted
          else:
            retweeted = "NA"
            
          # fixup search query to get topic
          topic = searchQuery[:searchQuery.find('-')].capitalize().strip()
          
          # fixup date
          tweetDate = tweet.created_at.date()

    
          
          tweet_lst.append([tweetDate, topic, 
                      tweet.id, tweet.user.screen_name, tweet.user.name, tweet.user.followers_count, tweet.text, tweet.favorite_count, 
                   tweet.retweet_count, retweeted, tweet.user.created_at, tweet.user.verified, tweet.user.location, tweet.user.statuses_count, tweet.created_at, tweet.retweeted, tweet.lang, tweet.coordinates])
        
        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break


print("Downloaded {0} tweets".format(tweetCount))

Downloaded 100 tweets
Downloaded 100 tweets


In [32]:
pd.set_option('display.max_colwidth', None)

# load it into a pandas dataframe
tweet_df = pd.DataFrame(tweet_lst, columns=['tweet_dt', 'topic', 'id', 'username', 'name', 'followers', 'tweet', 'like_count', 'retweet_count', 'retweeted', 'account_start_date', 'verified_user', 'location', 'number_of_tweets', 'creation_time', 'retweeted?', 'language', 'coordinates'])

tweet_df.head(5)

,tweet_dt,topic,id,username,name,followers,tweet,like_count,retweet_count,retweeted,account_start_date,verified_user,location,number_of_tweets,creation_time,retweeted?,language,coordinates
0,2020-09-18,Karachi,1306750526843949063,Stallion_Soul,HORSEMAN,205,"@KElectricPk And this is the height of dumbness, which clearly tells how the Customer Service operates without even… https://t.co/949lsqSDcq",0,0,False,2014-10-03 01:30:20,False,Mars,18083,2020-09-18 00:22:58,False,en,None
1,2020-09-18,Karachi,1306749722909003776,Tvvitterlndia,Chiraag,252,@omarali50 @TheseLongWars Is Shia population equally distributed all over Pakistan or it is mostly concentrated in… https://t.co/dVaNOXKs4F,0,0,False,2012-02-05 14:00:18,False,Yahi-Kahi,16415,2020-09-18 00:19:46,False,en,None
2,2020-09-18,Karachi,1306749721998839808,GVS_News,Global Village Space,18823,"In Pakistan, the conviction rate in rape and sexual assault cases is abysmally low, under 3%, estimates the Karachi… https://t.co/V6HJXcKjgQ",0,0,False,2016-11-01 09:03:07,False,,39265,2020-09-18 00:19:46,False,en,None
3,2020-09-18,Karachi,1306749447523753984,apniwebcompk,Apni Web News,133,Five injured after multi-storey building collapses in Karachi Korangi area https://t.co/IHCuhHMl4h,0,0,False,2017-08-12 04:16:50,False,Pakistan,127105,2020-09-18 00:18:41,False,en,None
4,2020-09-18,Karachi,1306749434202595328,VoK_021,Voice Of Karachi,227,New Train coaches for Karachi Circular Railway are ready. https://t.co/oMGCsg3hPh,1,0,False,2018-08-30 15:45:57,False,"کراچی, پاکستان",6759,2020-09-18 00:18:38,False,en,None
